#### Knjižnice za delo s podatki

In [1]:
import pandas as pd
import datetime as dt
import random
import math
from collections import defaultdict
import numpy as np
import itertools
from sklearn.metrics.pairwise import cosine_similarity

#### Branje Ocen (6)

In [2]:
class UserItemData:
    def __init__(self, path, start_date=None, end_date=None, min_ratings=0):
        self.path = path

        self.from_date = None
        self.to_date = None
        #Formatiranje datuma v format YYYY-MM-DD, da lahko primerjamo datume kot stringe
        if start_date:
            self.from_date = '-'.join([ (el if len(str(el)) > 1 else f'0{el}') for el in start_date.split('.')[::-1]])
        if end_date:
            self.to_date = '-'.join([ (el if len(str(el)) > 1 else f'0{el}') for el in end_date.split('.')[::-1]])
        
        self.min_ratings = min_ratings
        self.ratings = self._load_data()

    def _load_data(self):
        df = pd.read_csv(self.path, sep='\t')

        #Nov stolpec date, ki je kombinacija stolpcev date_year, date_month, date_day v formatu YYYY-MM-DD tipa string za primerjanje z start_date in end_date
        df['date'] = pd.to_datetime(df.date_year*10000+df.date_month*100+df.date_day,format='%Y%m%d')

        #Filtriranje vrstice glede na start_date, end_date in min_ratings
        if self.from_date:
            df = df[(df['date'] >= self.from_date)]
        
        if self.to_date:
            df = df[df['date'] < self.to_date]

        if self.min_ratings > 0:
            df = df.groupby('movieID').filter(lambda x: len(x) >= self.min_ratings)
        
        return df
    
    def nratings(self):
        #Vrne število zapisov v dataframe-u
        return len(self.ratings)

In [3]:
#Test branja podatkov in izpisa števila zapisov
uim = UserItemData('data/user_ratedmovies.dat')
print(uim.nratings())

uim = UserItemData('data/user_ratedmovies.dat', start_date = '12.1.2007', end_date='16.2.2008', min_ratings=100)
print(uim.nratings())

855598
73584


#### Branje filmov (6)

In [4]:
class MovieData:
    def __init__(self, path):
        self.path = path
        self.movies = self._load_data()
    
    def _load_data(self):
        df = pd.read_csv(self.path, sep='\t', encoding='ISO-8859-1')
        return df

    def get_title(self, movie_id):
        #Vrni naslov filma glede na id filma
        return self.movies[self.movies['id'] == movie_id].title.values[0]

In [5]:
#Testna poizvedba naslova filma po id-ju
md = MovieData('data/movies.dat')
print(md.get_title(1))

Toy story


#### Naključni prediktor (6)

In [6]:
class RandomPredictor:
    def __init__(self, min, max):
        self.min = min
        self.max = max
        self.data = None
    
    def fit(self, X):
        self.data = X

    def predict(self, user_id):
        #Za vsak film vrne naključno število med min in max
        return {(i+1):random.randint(self.min,self.max) for i in range(65133)}

In [7]:
#Izpis "napovedi" za uporabnika 78 za filme 1, 3, 20, 50, 100
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Film: {}, ocena: {}".format(md.get_title(item), pred[item]))

<class 'dict'>
Film: Toy story, ocena: 2
Film: Grumpy Old Men, ocena: 4
Film: Money Train, ocena: 5
Film: The Usual Suspects, ocena: 1
Film: City Hall, ocena: 5


#### Priporočanje (6)

In [8]:
class Recommender:
    def __init__(self, predictor):
        self.predictor = predictor

    def fit(self, X):
        self.predictor.fit(X)

    def recommend(self, user_id, n=10, rec_seen=True):
        #Iz predictorja pridobi slovar z napovedmi, ga uredi po oceni in vrne prvih n elementov
        pred = self.predictor.predict(user_id)
        pred = sorted(pred.items(), key=lambda x: x[1], reverse=True)

        #Če rec_seen == False, potem odstrani filme, ki jih je uporabnik že ocenil
        if not rec_seen:
            seen = self.predictor.data.ratings[self.predictor.data.ratings['userID'] == user_id].movieID.values
            pred = [el for el in pred if el[0] not in seen]

        return pred[:n]

In [9]:
#Izpis priporočil 5 filmov za uporabnika 78
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: Grumpy Old Men, ocena: 5
Film: Sabrina, ocena: 5
Film: Casino, ocena: 5
Film: Four Rooms, ocena: 5
Film: Persuasion, ocena: 5


#### Napovedovanje s povprečjem (6)

In [10]:
class AveragePredictor:
    def __init__(self, b):
        #b je parameter, ki se uporablja pri izračunu povprečja
        self.b = b if b>=0 else 0
        self.data = None
        self.predictions = None

    def fit(self, X):
        self.data = X
        df = X.ratings
        mean = df["rating"].mean()
        #Izračun povprečja za vsak film po formuli: avg = (vsota vseh ocen filma + b * povprečna ocena) / (število vseh ocen filma + b)
        self.predictions = {movie_id: (df[df["movieID"] == movie_id]["rating"].sum() + self.b * mean) / (len(df[df["movieID"]== movie_id]) + self.b) for movie_id in df['movieID'].unique()}

    def predict(self, user_id):
        #Vrne slovar z "napovedmi" (povprečnimi ocenami filma)
        return self.predictions
        

In [11]:
#Izpis priporočil 5 filmov za uporabnika 78 z AveragePredictor-ja, b = 0
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
ap = AveragePredictor(0)
rec = Recommender(ap)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: Brother Minister: The Assassination of Malcolm X, ocena: 5.0
Film: Synthetic Pleasures, ocena: 5.0
Film: Adam & Steve, ocena: 5.0
Film: Gabbeh, ocena: 5.0
Film: Eve and the Fire Horse, ocena: 5.0


In [12]:
#Izpis priporočil 5 filmov za uporabnika 78 z AveragePredictor-ja, b = 100
ap = AveragePredictor(100)
rec = Recommender(ap)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: The Usual Suspects, ocena: 4.225944245560473
Film: The Godfather: Part II, ocena: 4.146907937910189
Film: Cidade de Deus, ocena: 4.116538340205236
Film: The Dark Knight, ocena: 4.10413904093503
Film: 12 Angry Men, ocena: 4.103639627096175


#### Priporočanje najbolj gledanih filmov (6)

In [13]:
class ViewsPredictor:
    def __init__(self):
        self.data = None
        self.predictions = None

    def fit(self, X):
        self.data = X
        df = X.ratings
        #Izračun števila ocen za vsak film
        self.predictions = { group[0]: len(group[1]) for group in df.groupby('movieID')}

    def predict(self, user_id):
        return self.predictions

In [14]:
#Izpis priporočil 5 filmov za uporabnika 78 z ViewsPredictor-jem
vp = ViewsPredictor()
rec = Recommender(vp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 1576
Film: The Lord of the Rings: The Two Towers, ocena: 1528
Film: The Lord of the Rings: The Return of the King, ocena: 1457
Film: The Silence of the Lambs, ocena: 1431
Film: Shrek, ocena: 1404


#### Napovedovanje ocen s podobnostjo med produkti (6)

In [15]:
class ItemBasedPredictor:
    def __init__(self, min_values=0, threshold=0):
        self.min_values = min_values
        self.threshold = threshold
        self.data = None
        self.predictions = {}
        self.user_avgs = {}
        self.similarities = {}

    def fit(self, X):
        self.data = X
        df = X.ratings
        #Izračun povprečne ocene vsakega uporabnika
        self.user_avgs = { group[0]: group[1]['rating'].mean() for group in df.groupby('userID')}

        #Preoblikovanje podatkov v obliko kjer so vrstice movieID, stolpci userID in vrednosti v posameznem stolpcu so ocene uporabnika za dani film
        rm = df.pivot_table(index='movieID', columns='userID', values='rating')

        #Od vsake ocene odštejemo povprečno oceno uporabnika
        rm_norm = rm - rm.mean()

        #Za kombinacijo filmov i in j izračunamo podobnost med njima
        for i in rm.index:
            for j in rm.index:
                if i < j:
                    #Izračun podobnosti po popravljeni kosinusni razdalji
                    self.similarities[(i,j)] = (rm_norm.loc[i]* rm_norm.loc[j]).sum() / (math.sqrt((rm_norm.pow(2).loc[i]).where(rm_norm.loc[j].notnull()).sum()) * math.sqrt((rm_norm.pow(2).loc[j]).where(rm_norm.loc[i].notnull()).sum()))
                    
                    #Če je podobnost manjša od threshold, negativna ali ni zadostno število ocen, nastavi podobnost na 0
                    if self.similarities[(i,j)] < self.threshold or self.similarities[(i,j)] < 0  or len(rm_norm.loc[j].where(rm_norm.loc[i].notnull())) < self.min_values:
                        self.similarities[(i,j)] = 0
                
    def predict(self, user_id):
        #(movie, vsota(ocena *podobnost), vsota(podobnost))
        vsote1 = defaultdict(int)
        vsote2 = defaultdict(int)

        #Seznam filmov, ki jih je uporabnik že ocenil
        movie_ids = self.data.ratings[self.data.ratings['userID'] == user_id]['movieID'].unique()

        #Gremo čez vse pare filmo
        for (i,j), v in self.similarities.items():
            #Če je uporabnik ocenil film i vsoti prištejemo oceno filma i in podobnost med filmoma i in j, prav tako povečamo vstoto2 za podobnost med filmova
            if i in movie_ids:
                vsote1[j] += self.data.ratings[(self.data.ratings['userID'] == user_id) & (self.data.ratings['movieID'] == i)].rating.values[0] * v
                vsote2[j] += v
            #Enako za film j
            if j in movie_ids:
                vsote1[i] += self.data.ratings[(self.data.ratings['userID'] == user_id) & (self.data.ratings['movieID'] == j)].rating.values[0] * v
                vsote2[i] += v
        
        #Za vsak film oceno po formuli: vsota(ocena *podobnost) / vsota(podobnost)
        return { k: (v / vsote2[k]) if vsote2[k] != 0 else 0 for k, v in vsote1.items() if k not in movie_ids}

    def similarity(self, i, j):
        #Vrne podnosnost med filmoma i in j
        #Ker je podobnost med i in j enaka podobnosti med j in i, če je i > j, zamenjamo i in j
        if i > j:
            i,j = j,i
        return self.similarities[(i,j)] if (i,j) in self.similarities else 0

    def most_similar(self, n=20):
        #Uredi slovar podobnosti po vrednosti in vrni prvih n elementov
        return sorted(self.similarities.items(), key=lambda x: x[1], reverse=True)[:n]

    def similarItems(self, movie_id, n):
        #Med podobnostmi vzame tiste, ki imajo za i ali j movie_id, jih sortira po velikost in vrne prvih n elementov
        return sorted([(k[1], v) for k,v in self.similarities.items() if k[0] == movie_id] + [(k[0], v) for k,v in self.similarities.items() if k[1] == movie_id], key=lambda x: x[1], reverse=True)[:n]

        

In [32]:
#Izračun podobnosti za nekaj filmov
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)

print("Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780): ", rp.similarity(1580, 780))

Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716):  0.23395523176756633
Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527):  0
Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780):  0.4246612584468763


In [33]:
#Napoved ocen uporabnika za 15 filmov, ki napovedujemo, da bi jih najbolje ocenil
print("Predictions for 78: ")
rec_items = rec.recommend(78, n=15, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Predictions for 78: 
Film: Shichinin no samurai, ocena: 4.3557347903101595
Film: The Usual Suspects, ocena: 4.3546817280678365
Film: The Silence of the Lambs, ocena: 4.335305303472519
Film: Sin City, ocena: 4.2786871668991004
Film: Monsters, Inc., ocena: 4.2175811369435205
Film: The Incredibles, ocena: 4.207098583281748
Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 4.152792107348348
Film: Batman Begins, ocena: 4.146413806700199
Film: Die Hard, ocena: 4.125915602232819
Film: Rain Man, ocena: 4.07153524295855
Film: The Lord of the Rings: The Return of the King, ocena: 4.020237449257013
Film: A Beautiful Mind, ocena: 4.015142490064837
Film: Good Will Hunting, ocena: 4.009280806922821
Film: The Lord of the Rings: The Two Towers, ocena: 3.9414763050955943
Film: Indiana Jones and the Last Crusade, ocena: 3.7969764963789245


#### Najbolj podobni filmi (6)

In [18]:
#Izpis 20 medseboj najbolj podobnih filmov
for movie, val in rp.most_similar():
    print(f"Film 1: {md.get_title(movie[0])}, Film 2:{md.get_title(movie[1])}, podobnost: {val}")

Film 1: The Lord of the Rings: The Two Towers, Film 2:The Lord of the Rings: The Return of the King, podobnost: 0.8439842148481418
Film 1: The Lord of the Rings: The Fellowship of the Ring, Film 2:The Lord of the Rings: The Two Towers, podobnost: 0.8231885401761888
Film 1: The Lord of the Rings: The Fellowship of the Ring, Film 2:The Lord of the Rings: The Return of the King, podobnost: 0.8079374897442495
Film 1: Kill Bill: Vol. 2, Film 2:Kill Bill: Vol. 2, podobnost: 0.7372340224381029
Film 1: Star Wars, Film 2:Star Wars: Episode V - The Empire Strikes Back, podobnost: 0.7021321132220318
Film 1: Ace Ventura: Pet Detective, Film 2:The Mask, podobnost: 0.6616471778494046
Film 1: Star Wars: Episode V - The Empire Strikes Back, Film 2:Star Wars: Episode VI - Return of the Jedi, podobnost: 0.5992253753778948
Film 1: Independence Day, Film 2:Star Wars: Episode I - The Phantom Menace, podobnost: 0.5610426219249997
Film 1: Ace Ventura: Pet Detective, Film 2:Austin Powers: The Spy Who Shagged 

#### Priporočanje glede na trenutno ogledano vsebino (7)

In [19]:
#Izpis 10 najbolj podobnih filmov za film 'The Lord of the Rings: The Fellowship of the Ring'
rec_items = rp.similarItems(4993, 10)
print('Filmi podobni "The Lord of the Rings: The Fellowship of the Ring": ')
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Filmi podobni "The Lord of the Rings: The Fellowship of the Ring": 
Film: The Lord of the Rings: The Two Towers, ocena: 0.8231885401761888
Film: The Lord of the Rings: The Return of the King, ocena: 0.8079374897442495
Film: Star Wars: Episode V - The Empire Strikes Back, ocena: 0.23961943073496453
Film: Star Wars, ocena: 0.2196558652707407
Film: The Matrix, ocena: 0.2151555270688023
Film: Raiders of the Lost Ark, ocena: 0.19944276706345015
Film: The Usual Suspects, ocena: 0.18321188451910753
Film: Blade Runner, ocena: 0.1639968131541027
Film: Schindler's List, ocena: 0.16105905138148702
Film: Monty Python and the Holy Grail, ocena: 0.1578045379851914


#### Priporočilo zase (7)
Naredil sem kopijo datoteke user_ratedmovies.dat jo preimenoval user_ratedmovies.dat in na konec datoteke dodal zapise o ocenah filmov za userID sem si dodelil 1, saj so v datoteki uporabniki z ID-ji od 75 naprej.\
Zapisi:\
1	1	4	3	12	2007	2	56	3 \
1	2	5	3	12	2007	2	55	24 \
1	7	4	3	12	2007	2	38	12 \
1	47	4.5	3	12	2007	3	6	43 \
1	329	4	3	12	2007	3	1	37 \
1	479	4.5	3	12	2007	3	4	16 \
1	480	4.5	3	12	2007	2	34	18 \
1	761	4	10	10	2008	9	56	42 \
1	7361	4	3	12	2007	2	59	45 \
1	7371	2.5	3	12	2007	2	19	32 \
1	7438	4.5	3	12	2007	3	1	39 \
1	7579	4	3	12	2007	2	44	22 \
1	7934	3.5	3	12	2007	2	19	55 \
1	7983	5	3	12	2007	2	19	0 \
1	8638	4.5	3	12	2007	2	56	6 \
1	27727	3.5	3	12	2007	2	19	10 \
1	33166	4.5	3	12	2007	3	1	49 \
1	33564	4	3	12	2007	3	6	52 \
1	35836	0.5	3	12	2007	2	44	42 \
1	39869	4.5	3	12	2007	2	20	16 \
1	40629	2	3	12	2007	2	41	48 \
1	40819	4	3	12	2007	2	57	58 \
1	41285	3.5	3	12	2007	2	37	23 \
1	42900	4	3	12	2007	2	20	1 \
1	44555	4	3	12	2007	3	5	38 \
1	46578	4	3	12	2007	2	56	44 \
1	48516	4.5	3	12	2007	2	53	46 \
1	61075	5	10	10	2008	9	56	5 \
1	62049	4.5	10	10	2008	9	58	10 

In [29]:
#Ocenil sem nekaj filmov in jih dodal v kopijo datoteke na koncu z userID = 1
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies_medved.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)

In [30]:
#Napoved ocen uporabnika za 15 filmov, ki napovedujemo, da bi jih najbolje ocenil
print("Predictions for Medved(1): ")
rec_items = rec.recommend(1, n=10, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Predictions for Medved(1): 
Film: Braveheart, ocena: 4.5
Film: Forrest Gump, ocena: 4.5
Film: Terminator 2: Judgment Day, ocena: 4.5
Film: Die Hard, ocena: 4.5
Film: Star Wars: Episode VI - Return of the Jedi, ocena: 4.5
Film: Terminator Salvation, ocena: 4.5
Film: Indiana Jones and the Last Crusade, ocena: 4.5
Film: The Fifth Element, ocena: 4.5
Film: Rain Man, ocena: 4.5
Film: Saving Private Ryan, ocena: 4.5


Med priporočili je kar nekaj filmov npr Die Hard, Star Wars, Terminator, ki jih nisem gledal, a jih imam že nekaj časa na seznamu za ogled. Mogoče je to znak da se jih končno lotim.

#### Napovedovanje z metodo Slope one (7)

In [22]:
class SlopeOnePredictor:
    def __init__(self):
        self.data = None
        self.predictions = {}
        self.dev = {}
        self.weight = {}

    def fit(self, X):
        self.data = X
        df = X.ratings
        
        #Preoblikovanje tabele v obliko movieID x userID, kjer je vrednost med stolpcem in vrstico uporabnikova ocena filma
        rm = df.pivot_table(index='movieID', columns='userID', values='rating')

        #Loop čez kombinacijo vseh filmov
        for i in rm.index:
            for j in rm.index:
                if i != j:
                    #Izračun povprečne razlike med ocen kombinacije filov in števila uporabnikov, ki so ocenili oba filma
                    self.weight[(i,j)] = rm.loc[i].where(rm.loc[j].notnull()).count()
                    self.dev[(i,j)] = (rm.loc[i].where(rm.loc[j].notnull()) - rm.loc[j].where(rm.loc[i].notnull())).mean()
                    
                
    def predict(self, user_id):

        vsote1 = defaultdict(int)
        vsote_weight = defaultdict(int)
        #Seznam vseh filmov, ki jih je uporabnik ocenil
        movie_ids = self.data.ratings[self.data.ratings['userID'] == user_id]['movieID'].unique()

        #Loop čez vse povprečne razlike ocen parov filmov
        for (i,j), v in self.dev.items():
            #Če je uporabnik ocenil film j, ga upoštevamo pri napovedovanju ocene filma i
            if j in movie_ids:
                #Vsoti za film i prištejemo (user_rating + razlika med ocenama) * utež
                vsote1[i] += (self.data.ratings[(self.data.ratings['userID'] == user_id) & (self.data.ratings['movieID'] == j)].rating.values[0] + v) * self.weight[(i,j)]
                #Za vsak film seštevamo uteži
                vsote_weight[i] += self.weight[(i,j)]
        
        #Za vsak film vrnemo vsoto in jo delimo z vsoto uteži
        return { k: (v / vsote_weight[k]) for k, v in vsote1.items() }

In [23]:
#Priporočila 15 filmov (in napoved njihove ocene) za uporabnika 78 z uporabo SlopeOne prediktorja
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = SlopeOnePredictor()
rec = Recommender(rp)
rec.fit(uim)

print("Predictions for 78: ")
rec_items = rec.recommend(78, n=15, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Predictions for 78: 
Film: The Usual Suspects, ocena: 4.325079182263173
Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 4.155293229840448
Film: The Lord of the Rings: The Return of the King, ocena: 4.153135076202185
Film: The Silence of the Lambs, ocena: 4.127978169643881
Film: Shichinin no samurai, ocena: 4.119790444913598
Film: The Lord of the Rings: The Two Towers, ocena: 4.083325894849594
Film: Indiana Jones and the Last Crusade, ocena: 3.9670398355464194
Film: The Incredibles, ocena: 3.9664496674557546
Film: Good Will Hunting, ocena: 3.963362387354114
Film: Sin City, ocena: 3.942619137615212
Film: Batman Begins, ocena: 3.9375326640077017
Film: A Beautiful Mind, ocena: 3.9140940935239508
Film: Rain Man, ocena: 3.9107819079644943
Film: Monsters, Inc., ocena: 3.8819375978658006
Film: Finding Nemo, ocena: 3.8807711131654794


#### Metoda evaluate(self, test_data, n) (8)

In [24]:
#Razredu Recommender dodamo metodo evaluate, ki vrne rmse, mae, precision, recall in f1
class Recommender(Recommender):
    def evaluate(self, test_data, n=10):
        rmse = 0
        mae = 0
        precision = 0
        recall = 0
        count = 0
        users = test_data.ratings.userID.unique()
        
        
        for i in users:
            #Za vsakega uporabnika naredimo n napovedi
            rec_items = dict(self.recommend(i, n=n, rec_seen=False))
            #Iz testnih podatkov izberemo podatke za uporabnik in preoblikujemo dataframe, da lahko za posamezen film pogledamo oceno
            test_items = test_data.ratings[test_data.ratings['userID'] == i][['movieID', 'rating']].pivot_table(index='movieID', values='rating')
            
            #Za precision pogledamo, koliko napovedanih filmov si je uporabnik ogledal in delimo z številom priporočil
            precision += len(set(rec_items.keys()).intersection(set(test_items.index))) / n
            #Za recall pogledamo, koliko napovedanih filmov si je uporabnik ogledal in delimo z številom filmov, ki si jih je uporabnik "na novo" ogledal
            recall += len(set(rec_items.keys()).intersection((set(test_items.index)))) / len(set(test_items.index) - set(self.predictor.data.ratings[self.predictor.data.ratings['userID'] == i]['movieID'].unique()))

            #Za filme katere je uporabnik dobil napoved in jih pogledal/ocenil izračunamo napako ter jo prištejemo k vsoti napak
            for id in set(rec_items.keys()).intersection(set(test_items.index)):
                rmse += (test_items.loc[id][0] - rec_items[id]) ** 2
                mae += abs(test_items.loc[id][0] - rec_items[id])
                count += 1
        

        #Končni izračun metrik
        rmse = (rmse / count) ** 0.5
        mae = mae / count
        precision /= len(users)
        recall /= len(users)
        f = 2*precision*recall/(precision+recall)

        #Vrnitev rezultatov
        return (rmse,mae, precision, recall, f)


In [34]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000, end_date='1.1.2008')
rp = SlopeOnePredictor()
rec = Recommender(rp)
rec.fit(uim)

uim_test = UserItemData('data/user_ratedmovies.dat', min_ratings=200, start_date='2.1.2008')
mse, mae, precision, recall, f = rec.evaluate(uim_test, 20)
print(mse, mae, precision, recall, f)

0.8593687090470131 0.6321782795132675 0.04849690539345704 0.08567838257795103 0.061935941811173796


Najboljša priporočila dobim z Average prediktorjem, ki ima najmanjše napake in skoraj najvišje točke pri precision, recall, f1 score-ih. \
Napake SlopeOne prediktorja so zelo blizu Average prediktorja, a uporabnik le redko oceni filme, ki jih predlaga SlopeOne prediktor. \
Zanimiv rezultat je tudi pri views prediktorju, tu sicer ne moremo gledati napak, saj ne napovedujemo ocene, ampak ima najvišji f1 score. Zanimivo bi bilo pogledati ali so bolj gledani filmi tudi bolj ocenjeni ali pa ljudje samo bolj verjetno pogledamo filme, ki so jih pogledali tudi drugi. \
Najslabši izmed prediktorjev pa je, kot bi lahko pričakovali naključni prediktor.